In [1]:
import sys, pandas as pd
sys.executable, pd.__version__


('C:\\Users\\KIIT0001\\Desktop\\campus_companion\\attendance_ml\\.venv\\Scripts\\python.exe',
 '2.3.3')

In [2]:
df = pd.read_csv("../data/raw_attendance.csv")
df.head()


,student_id,date,subject,present,dept,semester
0,101,2024-01-01,Math,1,CSE,4
1,101,2024-01-02,Math,1,CSE,4
2,101,2024-01-03,Math,0,CSE,4
3,101,2024-01-04,Math,1,CSE,4
4,101,2024-01-05,Math,1,CSE,4


In [3]:
# Convert date column to datetime and sort
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["student_id", "date"]).reset_index(drop=True)

df.head()


,student_id,date,subject,present,dept,semester
0,101,2024-01-01,Math,1,CSE,4
1,101,2024-01-02,Math,1,CSE,4
2,101,2024-01-03,Math,0,CSE,4
3,101,2024-01-04,Math,1,CSE,4
4,101,2024-01-05,Math,1,CSE,4


In [4]:
# Group by student
g = df.groupby("student_id")

# Total classes and total presents per student
df["total_classes"] = g["present"].transform("count")
df["total_present"] = g["present"].transform("sum")

# Overall attendance %
df["overall_att"] = df["total_present"] / df["total_classes"]

df[["student_id", "date", "present", "overall_att"]].head(12)


,student_id,date,present,overall_att
0,101,2024-01-01,1,0.8
1,101,2024-01-02,1,0.8
2,101,2024-01-03,0,0.8
3,101,2024-01-04,1,0.8
4,101,2024-01-05,1,0.8
5,101,2024-01-06,1,0.8
6,101,2024-01-07,0,0.8
7,101,2024-01-08,1,0.8
8,101,2024-01-09,1,0.8
9,101,2024-01-10,1,0.8


In [5]:
# Rolling attendance for last 7 and last 30 classes
df["last7"] = g["present"].apply(
    lambda s: s.rolling(window=7, min_periods=1).mean()
)

df["last30"] = g["present"].apply(
    lambda s: s.rolling(window=30, min_periods=1).mean()
)

df[["student_id", "date", "present", "last7", "last30"]].head(12)


TypeError: incompatible index of inserted column with frame index

In [6]:
# Make sure groupby object exists
g = df.groupby("student_id")

# Rolling attendance for last 7 and last 30 classes
df["last7"] = g["present"].transform(
    lambda s: s.rolling(window=7, min_periods=1).mean()
)

df["last30"] = g["present"].transform(
    lambda s: s.rolling(window=30, min_periods=1).mean()
)

df[["student_id", "date", "present", "last7", "last30"]].head(12)


,student_id,date,present,last7,last30
0,101,2024-01-01,1,1.000000,1.000000
1,101,2024-01-02,1,1.000000,1.000000
2,101,2024-01-03,0,0.666667,0.666667
3,101,2024-01-04,1,0.750000,0.750000
4,101,2024-01-05,1,0.800000,0.800000
5,101,2024-01-06,1,0.833333,0.833333
6,101,2024-01-07,0,0.714286,0.714286
7,101,2024-01-08,1,0.714286,0.750000
8,101,2024-01-09,1,0.714286,0.777778
9,101,2024-01-10,1,0.857143,0.800000


In [7]:
def compute_absent_streak(s):
    streak = 0
    out = []
    for v in s:
        if v == 0:
            streak += 1
        else:
            streak = 0
        out.append(streak)
    return pd.Series(out, index=s.index)

# Apply per student
df["streak"] = g["present"].apply(compute_absent_streak)

df[["student_id", "date", "present", "streak"]].head(12)


TypeError: incompatible index of inserted column with frame index

In [8]:
# Recreate groupby just to be safe
g = df.groupby("student_id")

def compute_absent_streak(s):
    streak = 0
    out = []
    for v in s:
        if v == 0:
            streak += 1   # absent → streak++
        else:
            streak = 0    # present → streak reset
        out.append(streak)
    # IMPORTANT: sirf list return karo, Series nahi
    return out

# IMPORTANT: apply nahi, transform use karo
df["streak"] = g["present"].transform(compute_absent_streak)

df[["student_id", "date", "present", "streak"]].head(20)


,student_id,date,present,streak
0,101,2024-01-01,1,0
1,101,2024-01-02,1,0
2,101,2024-01-03,0,1
3,101,2024-01-04,1,0
4,101,2024-01-05,1,0
5,101,2024-01-06,1,0
6,101,2024-01-07,0,1
7,101,2024-01-08,1,0
8,101,2024-01-09,1,0
9,101,2024-01-10,1,0


In [9]:
# Week number
df["week"] = df["date"].dt.isocalendar().week

# Weekly attendance per student
weekly = (
    df.groupby(["student_id", "week"])["present"]
      .mean()
      .reset_index()
      .rename(columns={"present": "week_att"})
)

# Previous week's attendance
weekly["prev_week_att"] = (
    weekly.groupby("student_id")["week_att"].shift(1)
)

# Trend = this week - previous week
weekly["trend"] = weekly["week_att"] - weekly["prev_week_att"]

weekly


,student_id,week,week_att,prev_week_att,trend
0,101,1,0.714286,NaN,NaN
1,101,2,1.000000,0.714286,0.285714
2,102,1,0.285714,NaN,NaN
3,102,2,0.333333,0.285714,0.047619
4,103,1,1.000000,NaN,NaN
5,103,2,0.666667,1.000000,-0.333333


In [10]:
df = df.merge(
    weekly[["student_id", "week", "trend"]],
    on=["student_id", "week"],
    how="left"
)

df[["student_id", "date", "present", "week", "trend"]].head(12)


,student_id,date,present,week,trend
0,101,2024-01-01,1,1,NaN
1,101,2024-01-02,1,1,NaN
2,101,2024-01-03,0,1,NaN
3,101,2024-01-04,1,1,NaN
4,101,2024-01-05,1,1,NaN
5,101,2024-01-06,1,1,NaN
6,101,2024-01-07,0,1,NaN
7,101,2024-01-08,1,2,0.285714
8,101,2024-01-09,1,2,0.285714
9,101,2024-01-10,1,2,0.285714


In [11]:
df_latest = (
    df.sort_values("date")
      .groupby("student_id")
      .tail(1)
      .reset_index(drop=True)
)

df_latest[["student_id", "date", "overall_att", "last7", "last30", "streak", "trend"]]


,student_id,date,overall_att,last7,last30,streak,trend
0,102,2024-01-10,0.3,0.285714,0.3,1,0.047619
1,101,2024-01-10,0.8,0.857143,0.8,0,0.285714
2,103,2024-01-10,0.9,0.857143,0.9,0,-0.333333


In [12]:
# Threshold 0.75 (75% attendance)
df_latest["label"] = (df_latest["overall_att"] < 0.75).astype(int)

df_latest[["student_id", "overall_att", "last7", "streak", "trend", "label"]]


,student_id,overall_att,last7,streak,trend,label
0,102,0.3,0.285714,1,0.047619,1
1,101,0.8,0.857143,0,0.285714,0
2,103,0.9,0.857143,0,-0.333333,0


In [13]:
df_latest.to_csv("../data/processed_attendance.csv", index=False)
df_latest


,student_id,date,subject,present,dept,semester,total_classes,total_present,overall_att,last7,last30,streak,week,trend,label
0,102,2024-01-10,Math,0,CSE,4,10,3,0.3,0.285714,0.3,1,2,0.047619,1
1,101,2024-01-10,Math,1,CSE,4,10,8,0.8,0.857143,0.8,0,2,0.285714,0
2,103,2024-01-10,Math,1,CSE,4,10,9,0.9,0.857143,0.9,0,2,-0.333333,0
